In [ ]:
from fast import *
init_printing()
use_unicode=True; use_unicode=False

In [ ]:
from sympy import sin,cos,exp,sqrt,pi,zeros,I

We define the number of states and of radiation fields.

In [ ]:
Ne=2
Nl=1

We define the variables related to the laser field.

In [ ]:
E0,omega_laser=define_laser_variables(Nl)
pprint(E0,use_unicode=use_unicode)

In [ ]:
pprint(omega_laser,use_unicode=use_unicode)

We define a few important symbols.

In [ ]:
t,hbar,e=symbols("t hbar e",positive=True)
pprint([t,hbar,e],use_unicode=use_unicode)

We write an electric field propagating trough the $\hat{x}$ direction polarized in the $\hat{z}$ direction. First the wave vector:

In [ ]:
phi=0; theta=pi/2; alpha=pi/2; beta=0

k=Matrix([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])
pprint(k,use_unicode=use_unicode)

The polarization vectors.

In [ ]:
ep=polarization_vector(phi,theta,alpha,beta, 1)
em=polarization_vector(phi,theta,alpha,beta,-1)
pprint([ep,em],use_unicode=use_unicode)

The electric field (evaluated in $\vec{R}=0$).

In [ ]:
E_cartesian=(E0[0]/2*ep*exp(-I*omega_laser[0]*t) + E0[0].conjugate()/2*em*exp( I*omega_laser[0]*t))
pprint(E_cartesian,use_unicode=use_unicode)

We write the electric field in the helicity basis.

In [ ]:
E=cartesian_to_helicity(E_cartesian)
pprint(E,use_unicode=use_unicode)

We define the position operator.

In [ ]:
r=define_r_components(Ne,helicity=True,explicitly_hermitian=True)
pprint(r,use_unicode=use_unicode)

The frequencies of the energy levels, the resonant frequencies, and the decay frequencies.

In [ ]:
omega_level,omega,gamma=define_frequencies(Ne,explicitly_antisymmetric=True)
pprint(omega_level,use_unicode=use_unicode)

In [ ]:
pprint(omega,use_unicode=use_unicode)

In [ ]:
pprint(gamma,use_unicode=use_unicode)

The atomic hamiltonian is

In [ ]:
H0=Matrix([[hbar*omega_level[i]*KroneckerDelta(i,j) for j in range(Ne)] for i in range(Ne)])
pprint(H0,use_unicode=use_unicode)

The interaction hamiltonian is

In [ ]:
H1=e*helicity_dot_product(E,r)
pprint(H1,num_columns=120,use_unicode=use_unicode)

and the complete hamiltonian is

In [ ]:
H=H0+H1
pprint(H,num_columns=120,use_unicode=use_unicode)

# Rotating wave approximation
Notice that the electric field can be separated by terms with positive and negative frequency:

In [ ]:
E_cartesian_p=E0[0]            /2*ep*exp(-I*omega_laser[0]*t)
E_cartesian_m=E0[0].conjugate()/2*em*exp( I*omega_laser[0]*t)

E_p=cartesian_to_helicity(E_cartesian_p)
E_m=cartesian_to_helicity(E_cartesian_m)

pprint([E_p,E_m],use_unicode=use_unicode)

In [ ]:
pprint( simplify(E-(E_p+E_m)) ,use_unicode=use_unicode)

The position operator can also be separated in this way. We go to the interaction picture (with $\hat{H}_0$ as the undisturbed hamiltonian)

In [ ]:
r_I=[ Matrix([[exp(I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)] for i in range(Ne)]) for p in range(3)]
pprint(r_I[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I[2],use_unicode=use_unicode)

Which can be decomposed as

In [ ]:
r_I_p=[ Matrix([[ delta_greater(j,i)*exp(-I*omega[j,i]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_p[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I_p[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I_p[2],use_unicode=use_unicode)

In [ ]:
r_I_m=[ Matrix([[ delta_lesser( j,i)*exp( I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_m[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I_m[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I_m[2],use_unicode=use_unicode)

that summed equal $\vec{\hat{r}}_I$

In [ ]:
pprint( [r_I[p]-(r_I_p[p]+r_I_m[p]) for p in range(3)] ,use_unicode=use_unicode)

Thus the interaction hamiltonian in the interaciton picture is
\begin{equation}
    \hat{H}_{1I}=e\vec{E}\cdot \vec{\hat{r}}_I= e(\vec{E}^{(+)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(-)}_I)
\end{equation}

In [ ]:
H1I=e*helicity_dot_product(E,r_I)
pprint(H1I,num_columns=120,use_unicode=use_unicode)

Since both $\omega^l$ and $\omega_{ij}$ are in the order of THz, the terms that have frequencies with the same sign are summed, and thus also of the order of THz. The frequencies in the terms with oposite signs however, are detunings of the order of MHz. Since we are only interested in the coarse-grained evolution of the density matrix, we may omit the fast terms and approximate

\begin{equation}
    \hat{H}_{1I} \simeq \hat{H}_{1I,RWA}= e( \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I )
\end{equation}

That is known as the rotating wave approximation (RWA).

In [ ]:
H1IRWA=e*(helicity_dot_product(E_p,r_I_m)+helicity_dot_product(E_m,r_I_p))
pprint(H1IRWA,use_unicode=use_unicode)

 Returning to the Schrödinger picture we have.

In [ ]:
r_p=[ Matrix([[ delta_greater(j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_p,use_unicode=use_unicode)

In [ ]:
r_m=[ Matrix([[ delta_lesser( j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_m,use_unicode=use_unicode)

In [ ]:
pprint( [r[p]-(r_p[p]+r_m[p]) for p in range(3)] ,use_unicode=use_unicode)

Thus the interaction hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [ ]:
H1RWA=e*(helicity_dot_product(E_p,r_m)+helicity_dot_product(E_m,r_p))
pprint(H1RWA,use_unicode=use_unicode)

And the complete hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [ ]:
HRWA=H0+H1RWA
pprint(HRWA,use_unicode=use_unicode)

# Rotating Frame
Next we will make a phase transformation in order to eliminate the explicit time dependance of the equations.

In [ ]:
c,ctilde,phase=define_psi_coefficients(Ne)
pprint([c,ctilde,phase],use_unicode=use_unicode)

In [ ]:
psi=Matrix([ exp(I*phase[i]*t)*ctilde[i] for i in range(Ne)])
pprint(psi,use_unicode=use_unicode)

The Schrödinger equation $i\hbar \partial_t |\psi\rangle=\hat{H}_{RWA}$ is

In [ ]:
lhs=Matrix([(I*hbar*Derivative(psi[i],t).doit()).expand() for i in range(Ne)])
pprint(lhs,use_unicode=use_unicode)

In [ ]:
rhs=HRWA*psi
pprint(rhs,num_columns=120,use_unicode=use_unicode)

We multiply each of these equations by $e^{-i \theta_i t}$ and substracting $i \theta_i \tilde{c}_i$

In [ ]:
lhs_new=Matrix([simplify(  lhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(lhs_new,use_unicode=use_unicode)

In [ ]:
rhs_new=Matrix([simplify(  rhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(rhs_new,num_columns=120,use_unicode=use_unicode)

It can be seen that the equations loose their explicit time dependance only if $\omega^{1} - \theta_{1} + \theta_{2}=0$. Which is satisfied if

In [ ]:
phase_transformation=solve(omega_laser[0]+phase[1]-phase[0],phase[1],dict=True)[0]
pprint(phase_transformation,use_unicode=use_unicode)

There is a free parameter $\theta_1$, which is to be expected, since state vetors $|\psi\rangle$ always have a global phase invariance

In [ ]:
pprint(psi.subs(phase_transformation),use_unicode=use_unicode)

Thus the equations become

In [ ]:
pprint(lhs_new,use_unicode=use_unicode)

In [ ]:
rhs_new=simplify(rhs_new.subs(phase_transformation))
pprint(rhs_new,use_unicode=use_unicode)

It can be seen that this is the Schrödinger equation derived from an effective hamiltonian $\tilde{H}$

In [ ]:
Htilde=Matrix([ [Derivative(rhs_new[i],ctilde[j]).doit() for j in range(Ne)] for i in range(Ne)])
pprint(Htilde,use_unicode=use_unicode)

We can see that it is convenient to choose $\theta_1=-\omega_1$ to simplify the hamiltonian. Also, we can recognize $\omega^1-\omega_2+\omega_1=\delta$ as the detuning of the laser field relative to the atomic transition $\omega_{21}=\omega_2-\omega_1$.

In [ ]:
delta=Symbol("delta",real=True)
Htilde=Htilde.subs({phase[0]:-omega_level[0]}).subs({omega_laser[0]:delta+omega_level[1]-omega_level[0]})
pprint(Htilde,use_unicode=use_unicode)

If we define the Rabi frequency $\Omega =e E_0^1 r_{0;21}/\hbar$

In [ ]:
Omega=Symbol("Omega",real=False)
Htilde=Htilde.subs({E0[0]:Omega*hbar/r[1][1,0]/e})
pprint(Htilde,use_unicode=use_unicode)

We define the density matrix.

In [ ]:
rho=define_density_matrix(Ne)
pprint( rho ,use_unicode=use_unicode)

The hamiltonian part of the equations is
\begin{equation}
    \dot{\hat{\rho}}=\frac{i}{\hbar}[\hat{\rho}, \hat{\tilde{H}}]
\end{equation}

In [ ]:
hamiltonian_terms=(I/hbar*(rho*Htilde-Htilde*rho)).expand()
pprint(hamiltonian_terms,use_unicode=use_unicode)

There is only one Lindblad operator, since there is only one spontaneous decay channel.

In [ ]:
lindblad_terms=gamma[1,0]*lindblad_operator(ket(1,Ne)*bra(2,Ne),rho)
pprint(lindblad_terms, num_columns=120,use_unicode=use_unicode)

# Optical Bloch Equations
The Optical Bloch equations are thus.

In [ ]:
eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120,use_unicode=use_unicode)

which is how most literature will show the equations. However, a more convenient way to express this equations is to explicitly asume a normalized and hermitian density matrix

In [ ]:
rho=define_density_matrix(Ne,explicitly_hermitian=True,normalized=True)
pprint( rho ,use_unicode=use_unicode)

In [ ]:
hamiltonian_terms = (I/hbar*(rho*Htilde-Htilde*rho)).expand()
lindblad_terms    =gamma[1,0]*lindblad_operator(ket(1,Ne)*bra(2,Ne),rho)
eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120,use_unicode=use_unicode)

and only consider the equations for the populations $\rho_{ii}$ for $i>1$ and the real and imaginary parts of the coherences below the diagonal.

In [ ]:
ss_comp={ rho[i,j]:re(rho[i,j])+I*im(rho[i,j]) for j in range(Ne) for i in range(Ne)}
pprint( re(eqs[1,1].subs(ss_comp)) ,use_unicode=use_unicode)

In [ ]:
pprint( re(eqs[1,0].subs(ss_comp)) ,use_unicode=use_unicode)

In [ ]:
pprint( im(eqs[1,0].subs(ss_comp)) ,use_unicode=use_unicode)

If the density matrix is represented as a vector whose components are the these independent components of the density matrix

In [ ]:
rho_vect=define_rho_vector(rho,Ne)
pprint(rho_vect,use_unicode=use_unicode)

Then the equations can be re-written as linear combinations of these components plus an independent term.
\begin{equation}
    \dot{\vec{\rho}} = \hat{A} \vec{\rho} + \vec{b}
\end{equation}
with $\hat{A}$ a linear operator acting in this vector space and $\vec{b}$ the vector of independent terms.

In [ ]:
A,b=calculate_A_b(eqs,rho,Ne)
pprint([A,b],use_unicode=use_unicode)

Explicitly, this is

In [ ]:
eqs_new=A*rho_vect - b
pprint(eqs_new,use_unicode=use_unicode)

Which is the same as the equations in the previous form.

In [ ]:
pprint( eqs_new - Matrix([re(eqs[1,1]),re(eqs[1,0]),im(eqs[1,0])]).subs(ss_comp) ,use_unicode=use_unicode)

The steady state solution of this equations is

In [ ]:
sol=solve(list(eqs_new),list(rho_vect))
for mu in range(3):
    pprint( {rho_vect[mu]:sol[rho_vect[mu]]} ,num_columns=120,use_unicode=use_unicode)

According to literature [1], the solution should be

In [ ]:
s0=2*(re(Omega)**2+im(Omega)**2)/gamma[1,0]**2

s=s0/(1+(2*delta/gamma[1,0])**2)


rho21=-I*Omega/(2*(gamma[1,0]/2-I*delta)*(1+s))

rerho22=( s/(2*(1+s)) ).simplify()
rerho21=re(rho21).simplify()
imrho21=im(rho21).simplify()

test=[ sol[rho[1,1]]-rerho22, sol[re(rho[1,0])]-rerho21, sol[im(rho[1,0])]-imrho21 ]

pprint( [testi.subs({Omega:re(Omega)+I*im(Omega)}).factor() for testi in test] ,use_unicode=use_unicode)

So our development produces the same results as the literature.

The saturation intensity is defined as the intensity needed to accumulate $\frac{1}{4}$ of the population in the excited state when the field is in resonance ($\delta=0$).

In [ ]:
saturation_eq=sol[rho[1,1]].subs({delta:0})-1/Integer(4)
pprint( saturation_eq ,use_unicode=use_unicode)

In [ ]:
Omega_amp,alpha=symbols("\Omega_a alpha",real=True)
ss={Omega:Omega_amp*cos(alpha)+I*Omega_amp*sin(alpha)}
saturation_eq= saturation_eq.subs(ss).factor().simplify()
pprint(saturation_eq,use_unicode=use_unicode)

In [ ]:
Omega_sat=solve( saturation_eq ,Omega_amp)[1]
pprint(Omega_sat,use_unicode=use_unicode)

Since $\Omega =e E_0^1 r_{0;21}/\hbar$ it follows that

In [ ]:
E0_sat=Omega_sat*hbar/e/r[1][1,0]
pprint(E0_sat,use_unicode=use_unicode)

The full width at half maximum of $\rho_{22}$ is

In [ ]:
hm1,hm2=solve(sol[rho[1,1]]-sol[rho[1,1]].subs({delta:0})/2,delta)
FWHM=hm2-hm1
FWHM=FWHM.subs(ss).simplify()
pprint(FWHM,use_unicode=use_unicode)

[1]  H.J. Metcalf and P. van der Straten. Laser Cooling and Trapping. Graduate Texts in Contempo-
rary Physics. Springer New York, 2001.